In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os

os.environ["KAGGLE_CONFIG_DIR"] = '/content/drive/My Drive/Kaggle'

In [ ]:
%cd /content/drive/My\ Drive/Kaggle

/content/drive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d antoniuscs/imdb-synopsis-indonesian-movies

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 19.8MB/s]


In [ ]:
# !unzip imdb-synopsis-indonesian-movies.zip

Archive:  imdb-synopsis-indonesian-movies.zip
  inflating: imdb_indonesian_movies_2.csv  


In [ ]:
import pandas as pd

dataset = pd.read_csv('imdb_indonesian_movies_2.csv')
dataset

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...,...
1000,Winter in Tokyo,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Petualang Cinta,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,Last Night (III),"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Path of Light,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


In [ ]:
dataset = dataset.drop(columns=["judul_film"], axis=1)
dataset

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


In [ ]:
dummies = pd.get_dummies(dataset.genre)
dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=["genre"], axis=1)
dataset

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [ ]:
X = dataset.ringkasan_sinopsis
y = dataset.iloc[:, 1:]
y

,Drama,Horor,Komedi,Laga,Romantis
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
1000,0,0,0,0,1
1001,0,0,0,0,1
1002,0,0,0,0,1
1003,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2)

In [ ]:
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

sequence_train = tokenizer.texts_to_sequences(X_train)
sequence_test = tokenizer.texts_to_sequences(X_test)

pad_train = pad_sequences(sequence_train)
pad_test = pad_sequences(sequence_test)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          80000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 512)               33280     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 136,581
Trainable params: 136,581
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sgd = tf.keras.optimizers.SGD()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy")>0.99):
            print(" Reached 99% Accuracy")
            self.model.stop_training = True
            
callbacks = myCallback()

In [ ]:
from time import time

t1 = time()
history = model.fit(pad_train, y_train, epochs=30, validation_data=(pad_test, y_test), callbacks=[callbacks])

Epoch 1/30
26/26 [==============================] - 29s 1s/step - loss: 1.0745 - accuracy: 0.5522 - val_loss: 2.1056 - val_accuracy: 0.2189
Epoch 2/30
26/26 [==============================] - 29s 1s/step - loss: 0.7127 - accuracy: 0.7488 - val_loss: 2.3376 - val_accuracy: 0.2289
Epoch 3/30
26/26 [==============================] - 30s 1s/step - loss: 0.4816 - accuracy: 0.8495 - val_loss: 2.6830 - val_accuracy: 0.2040
Epoch 4/30
26/26 [==============================] - 31s 1s/step - loss: 0.2192 - accuracy: 0.9465 - val_loss: 3.5207 - val_accuracy: 0.2289
Epoch 5/30
26/26 [==============================] - 30s 1s/step - loss: 0.0870 - accuracy: 0.9838 - val_loss: 3.7374 - val_accuracy: 0.2388
Epoch 6/30
26/26 [==============================] - 30s 1s/step - loss: 0.0412 - accuracy: 0.9938 - val_loss: 4.3042 - val_accuracy: 0.2289


In [ ]:
print(f"Training time: {round(time()-t1)} Sec")

Training time: 186 Sec
